# Intermovie, la bibliothèque du 7ème art

In [ ]:
%load_ext autoreload
%autoreload 2

Import des librairies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functions import nettoyage as net

## Téléchargement et extraction des données

In [ ]:
from functions import download

##### Création de l'instance de classe #####
download_object = download.downloader("./data")

##### Téléchargement du fichier #####
download_object.data_download("data_file.zip", 'https://stdatalake005.blob.core.windows.net/public/movies_dataset.zip')

##### Extraction du fichier #####
download_object.data_extract("data_file.zip")

## Préparation des datasets

In [ ]:
from functions import split

##### Création de l'instance de classe #####
splitter_object = split.splitter("./data", '\t')

##### Split des fichiers #####
splitter_object.file_splitter("title.principals.tsv", "category", "title.principals")
splitter_object.file_splitter("title.akas.tsv", "region", "title.akas")
splitter_object.file_splitter("title.basics.tsv", "titleType", "title.basics")

## Création du premier CSV

In [2]:
data_title_clean = net.nettoyage_title_basics()

data_noms_clean = net.nettoyage_name_basics()

data_films_clean = net.nettoyage_title_principals()

#fusion des dataframe
data_films_clean.set_index('tconst', inplace  = True)
data_films_with_title = data_films_clean.join(data_title_clean).reset_index().drop(["tconst"], axis = 1)
del data_films_clean
del data_title_clean
data_films_with_title.set_index('nconst', inplace  = True)
data_final = data_films_with_title.join(data_noms_clean).reset_index().drop(["nconst"], axis = 1)
del data_films_with_title
del data_noms_clean

#convertion en csv
data_final = data_final.dropna()
data_final.to_csv("./data/OUTPUT/csv_req_1.csv")
del data_final

## Création du deuxième CSV

In [3]:
data_title_clean = net.nettoyage_title_basics()

data_ratings_clean = net.nettoyage_title_ratings()

data_natio_clean = net.nettoyage_title_akas()

#fusion des dataframe
data_natio_clean = data_natio_clean.join(data_title_clean)
data_final = data_natio_clean.join(data_ratings_clean).reset_index()
data_final.drop_duplicates(keep = 'first', inplace=True)
del data_natio_clean

#convertion en csv
data_final = data_final.dropna()
data_final.to_csv("./data/OUTPUT/csv_req_2.csv")
del data_final

## Création du troisième CSV

In [4]:
data_title_clean = net.lecture_title_basics()

data_ratings_clean = net.nettoyage_title_ratings()

#fusion des dataframes
df_intermediaire = data_title_clean.join(data_ratings_clean)
df_intermediaire = df_intermediaire.reset_index()
df_intermediaire = df_intermediaire.drop(["tconst"], axis = 1)
df_intermediaire = df_intermediaire.dropna()
del data_title_clean
del data_ratings_clean

#calcul moyenne
df_cat = pd.DataFrame
df_cat = df_intermediaire["genres"].str.split(",", expand=True)
df_intermediaire = df_intermediaire.join(df_cat).drop(["genres"], axis=1)
del df_cat
df_moy1 = df_intermediaire.groupby([0])["averageRating"].mean()
df_moy2 = df_intermediaire.groupby([1])["averageRating"].mean()
df_moy3 = df_intermediaire.groupby([2])["averageRating"].mean()
df_moy = pd.concat([df_moy1, df_moy2, df_moy3], axis = 1, keys = ["moy1", "moy2", "moy3"])
del df_intermediaire
del df_moy1
del df_moy2
del df_moy3
df_moy['moy'] = df_moy[['moy1', 'moy2', 'moy3']].mean(axis=1)
df_moy = df_moy.drop(["moy1", "moy2", "moy3"], axis = 1)

#convertion en csv
df_moy.to_csv('./data/OUTPUT/csv_req_3.csv')
del df_moy

# Création du quatrième CSV

In [5]:
data_title_clean = net.nettoyage_title_basics()

data_noms_clean = net.nettoyage_name_basics()

data_films_clean = net.nettoyage_title_principals()

data_ratings_clean = net.nettoyage_title_ratings()

#fusion des dataframes
data_films_clean.set_index('tconst', inplace  = True)
data_films_with_title = data_films_clean.join(data_title_clean)
data_films_with_title = data_films_with_title.join(data_ratings_clean).reset_index().drop(["tconst"], axis = 1)
del data_films_clean
del data_title_clean
del data_ratings_clean
data_films_with_title.set_index('nconst', inplace  = True)
data_final = data_films_with_title.join(data_noms_clean).reset_index().drop(["nconst"], axis = 1)
del data_films_with_title
del data_noms_clean

#calcul de la moyenne
data_final = data_final.dropna()
data_final = data_final.groupby(["primaryName"]).mean()

#conversion en csv
data_final.to_csv('./data/OUTPUT/csv_req_4.csv')
del data_final